In [1]:
#Tackle Detection Model

In [2]:
pip install tensorflow


  Using cached tensorflow-2.15.0-cp39-cp39-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.15.0-cp39-cp39-win_amd64.whl (300.8 MB)
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached grpcio-1.60.0-cp39-cp39-win_amd64.whl (3.7 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorboard-2.15.1-py3-none-any.whl (5.5 MB)
  Using cached google_auth-2.26.2-py2.py3-none-any.whl (186 kB)
  Using cached google_auth_oauthlib-1.2.0-py2.py3-none-any.whl (24 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip show tensorflow


Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\nehaa\anaconda3\lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [5]:
!pip list | grep tensorflow


tensorflow                    2.15.0
tensorflow-estimator          2.15.0
tensorflow-intel              2.15.0
tensorflow-io-gcs-filesystem  0.31.0


In [23]:
import tensorflow as tf


In [19]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models,layers


In [20]:
# Load and preprocess the dataset
def load_dataset(data_dir, target_size=(224, 224)):
    X, y = [], []

    for label, class_dir in enumerate(['tackles', 'non_tackles']):
        class_path = os.path.join(data_dir, class_dir)

        if not os.path.exists(class_path):
            print(f"Warning: Directory '{class_dir}' not found in {data_dir}. Skipping...")
            continue

        for video_dir in os.listdir(class_path):
            video_path = os.path.join(class_path, video_dir)

            if not os.path.exists(video_path):
                print(f"Warning: Video directory '{video_dir}' not found in {class_path}. Skipping...")
                continue

            for frame_file in os.listdir(video_path):
                frame_path = os.path.join(video_path, frame_file)

                # Load and preprocess the frame
                frame = load_img(frame_path, target_size=target_size)
                frame = img_to_array(frame) / 255.0

                X.append(frame)
                y.append(class_dir)

    if not X or not y:
        print("Error: No data found in the specified directory. Please check your dataset structure.")
        return None, None

    X = np.array(X)
    y = np.array(y)

    # Convert labels to numerical format
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    y = to_categorical(y)

    return X, y

# Create a ResNet50 model
def create_resnet50_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freezing  the pre-trained layers( to avoid training them during next steps)
    for layer in base_model.layers:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Replace 'path/to/your/dataset' with the actual path to your dataset
data_dir = r"C:\Users\nehaa\Downloads\frames of the dataset"

# Load and preprocess the dataset
X, y = load_dataset(data_dir)

# Check if the dataset is loaded successfully
if X is None or y is None:
    exit()



In [21]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a ResNet50 model
input_shape = X_train[0].shape
num_classes = 2  # Adjust based on your task (tackle and non-tackle)
model = create_resnet50_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')



Epoch 1/50


23/23 [==============================] - 63s 2s/step - loss: 0.6695 - accuracy: 0.6747 - val_loss: 0.5389 - val_accuracy: 0.7514
Epoch 2/50
23/23 [==============================] - 55s 2s/step - loss: 0.5591 - accuracy: 0.7355 - val_loss: 0.4952 - val_accuracy: 0.7514
Epoch 3/50
23/23 [==============================] - 54s 2s/step - loss: 0.5164 - accuracy: 0.7511 - val_loss: 0.4477 - val_accuracy: 0.8644
Epoch 4/50
23/23 [==============================] - 52s 2s/step - loss: 0.4676 - accuracy: 0.7992 - val_loss: 0.4194 - val_accuracy: 0.7514
Epoch 5/50
23/23 [==============================] - 54s 2s/step - loss: 0.4491 - accuracy: 0.7949 - val_loss: 0.3669 - val_accuracy: 0.8983
Epoch 6/50
23/23 [==============================] - 56s 2s/step - loss: 0.4159 - accuracy: 0.8246 - val_loss: 0.3412 - val_accuracy: 0.8983
Epoch 7/50
23/23 [==============================] - 55s 2s/step - loss: 0.3674 - accuracy: 0.8529 - val_loss: 0.3026 - val_accuracy: 0.8983
Epoch 8/50
23/23 

In [24]:
# Save the trained model to a file using the SavedModel format
tf.saved_model.save(model, r"C:\Users\nehaa\Downloads\Real-time Event recognition\saved_model")

# Load the saved model
loaded_model = tf.saved_model.load(r"C:\Users\nehaa\Downloads\Real-time Event recognition\saved_model")

# Now you can use loaded_model for predictions


INFO:tensorflow:Assets written to: C:\Users\nehaa\Downloads\Real-time Event recognition\saved_model\assets


INFO:tensorflow:Assets written to: C:\Users\nehaa\Downloads\Real-time Event recognition\saved_model\assets


In [26]:
import cv2
import time
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder



In [28]:
# Get the signature of the model(i.e inputs and outputs of the model)
signature = loaded_model.signatures["serving_default"]

# Replace 'path/to/your/random_video.mp4' with the actual path to your video
random_video_path = r"C:\Users\nehaa\Downloads\dataset for soccer\tackles\1.mp4"

# Load the video
cap = cv2.VideoCapture(random_video_path)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Resize the frame to match the input size of your model
    frame = cv2.resize(frame, (224, 224))

    # Preprocess the frame
    frame = frame / 255.0  # Normalize to [0, 1]
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension

    # Make a prediction using the model
    prediction = signature(tf.constant(frame, dtype=tf.float32))
    
    # Print the keys of the prediction dictionary
    print("Prediction Keys:", prediction.keys())

    # Assuming class 0 is 'tackle'
    output_key='dense_1'
    if prediction['dense_1'][0, 0] > 0.5:
        print("Tackle detected!")
    else:
        print("No tackle detected.")

    # Display the frame (optional)
    cv2.imshow('Frame', frame[0])

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])
No tackle detected.
Prediction Keys: dict_keys(['dense_1'])


In [29]:
pip install pafy

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install youtube_dl



     ---------------------------------------- 1.9/1.9 MB 61.5 kB/s eta 0:00:00


In [31]:
pip install pytube


  Using cached pytube-15.0.0-py3-none-any.whl (57 kB)
Note: you may need to restart the kernel to use updated packages.


In [35]:
import cv2
import numpy as np
import tensorflow as tf
import time
import subprocess

# pre-trained ResNet50 model
saved_model_path = r"C:\Users\nehaa\Downloads\Real-time Event recognition\saved_model" 
loaded_model = tf.saved_model.load(saved_model_path)



In [33]:
from pytube import YouTube


In [36]:
import cv2
import numpy as np
import time
from pytube import YouTube

def download_youtube_video(video_url, output_path='temp_video.mp4'):
    try:
        yt = YouTube(video_url)
        ys = yt.streams.get_highest_resolution()
        ys.download(output_path)
        return output_path
    except Exception as e:
        print(f"Error: {e}")
        return None

def get_youtube_frames(video_url, num_frames=300):
    try:
        yt = YouTube(video_url)
        ys = yt.streams.get_highest_resolution()
        cap = cv2.VideoCapture(ys.url)
        frames = []

        for _ in range(num_frames):
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)

        cap.release()
        return frames
    except Exception as e:
        print(f"Error: {e}")
        return None

def detect_tackle(frame):
    # Preprocess the input frame to match the model's expected input format
    frame = cv2.resize(frame, (224, 224))
    frame = frame / 255.0  # Normalize to [0, 1]
    frame = np.expand_dims(frame, axis=0)  # Add batch dimension
    frame = tf.convert_to_tensor(frame, dtype=tf.float32)  # Convert to TensorFlow tensor

    # Using the loaded model for prediction
    prediction = loaded_model(frame)

    return np.argmax(prediction) == 0  # Assuming class 0 is 'tackle'

def process_video_stream(video_source):
    # Get frames directly from the YouTube video
    frames = get_youtube_frames(video_source)

    if frames is None:
        print("Error: Could not retrieve frames from the YouTube video.")
        return

    # Variable to store the start time of the event
    event_start_time = None

    for frame in frames:
        # Perform tackle detection
        is_tackle = detect_tackle(frame)

        # If a tackle is detected, trigger the event
        if is_tackle:
            if event_start_time is None:
                event_start_time = time.time()
                print(f"Tackle detected! Event started at {event_start_time}")

        # Display the frame
        cv2.imshow('Soccer Match', frame)

        # Check for user input to exit the loop
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break

    # Close the display window
    cv2.destroyAllWindows()

    # If a tackle was detected, print the exact time
    if event_start_time is not None:
        event_end_time = time.time()
        elapsed_time = event_end_time - event_start_time

        # Convert elapsed time to minutes and seconds
        minutes, seconds = divmod(elapsed_time, 60)
        print(f"Event duration: {int(minutes)} minutes and {seconds:.2f} seconds")

# Replace 'your_video_source' with the actual YouTube video URL
process_video_stream(r"https://www.youtube.com/watch?v=wRAUnwpRfgk")



Tackle detected! Event started at 1705563773.1554792
Event duration: 0 minutes and 37.13 seconds
